In [1]:
import argparse
import os
import sys
from os import mkdir
from apex import amp
import shutil



import torch.nn.functional as F

sys.path.append('..')
from config import cfg
from data import make_data_loader, make_data_loader_view
from engine.trainer import do_train
from modeling import build_model
from solver import make_optimizer, WarmupMultiStepLR,build_scheduler
from layers import make_loss

from utils.logger import setup_logger

#from utils import vis_density

from torch.utils.tensorboard import SummaryWriter
import torch
from layers.RaySamplePoint import RaySamplePoint
from engine import render
import numpy as np 
torch.cuda.set_device(2)
import matplotlib.pyplot as plt
import cv2
from tqdm.notebook import trange, tqdm
from imageio_ffmpeg import write_frames
import trimesh

/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [2]:
def vis_density(model,bbox, L= 32):

    maxs = torch.max(bbox, dim=0).values
    mins = torch.min(bbox, dim=0).values


    x = torch.linspace(mins[0],maxs[0],steps=L).cuda()
    y = torch.linspace(mins[1],maxs[1],steps=L).cuda()
    z = torch.linspace(mins[2],maxs[2],steps=L).cuda()
    grid_x ,grid_y,grid_z = torch.meshgrid(x, y,z)
    xyz = torch.stack([grid_x ,grid_y,grid_z], dim = -1)  #(L,L,L,3)

    xyz = xyz.reshape((-1,3)) #(L*L*L,3)


    xyzs = xyz.split(5000, dim=0)

    sigmas = []
    for i in xyzs:
        with torch.no_grad():
            _,density = model.spacenet_fine(i, None, model.maxs, model.mins) #(L*L*L,1)
            density = torch.nn.functional.relu(density)
            sigmas.append(density.detach().cpu())

    sigmas = torch.cat(sigmas, dim=0)

    return sigmas,xyz

In [3]:
training_folder = '/data/wmy/NR/RF_training/panganqi_near_far_ori'
epoch = 834000
para_file = 'rfnr_model_%d.pth' % epoch

In [ ]:
training_folder = '/data/wmy/NR/RF_training/plant'
epoch = 714000
para_file = 'rfnr_model_%d.pth' % epoch

In [4]:
cfg.merge_from_file(os.path.join(training_folder,'configs.yml'))
cfg.MODEL.BOARDER_WEIGHT = 1e-10
cfg.freeze()

In [5]:
val_loader, dataset_val = make_data_loader_view(cfg, is_train=False)
train_loader, dataset = make_data_loader(cfg, is_train=True)

0 / 1
load 56 Ts, 56 Ks, 1 frame, 400876 vertices
dataset initialed. near: 3.008136  far: 12.583911
0 / 1
load 56 Ts, 56 Ks, 1 frame, 400876 vertices
dataset initialed. near: 3.008136  far: 12.583911
load 25684800 rays.


In [6]:
model = build_model(cfg).cuda()
model.load_state_dict(torch.load(os.path.join(training_folder,para_file),map_location='cpu'))
model.eval()
model.cuda()
pass

In [7]:
dataset.bbox[0]

tensor([[ -0.5053,  -6.6616, -10.9217],
        [  1.0823,  -6.6616, -10.9217],
        [  1.0823,  -1.7790, -10.9217],
        [ -0.5053,  -1.7790, -10.9217],
        [ -0.5053,  -6.6616,  -9.1911],
        [  1.0823,  -6.6616,  -9.1911],
        [  1.0823,  -1.7790,  -9.1911],
        [ -0.5053,  -1.7790,  -9.1911]])

In [17]:
L = 512
sigma,xyzs = vis_density(model, dataset.bbox[0], L)

In [19]:
raw = sigma.detach().cpu().numpy()
raw_xyz = xyzs.detach().cpu().numpy()
raw = np.reshape(raw, (L,L,L,-1))
raw_xyz = np.reshape(raw_xyz, (L,L,L,-1))
sigma = np.maximum(raw[...,-1], 0.)

print(sigma.shape)
print(raw_xyz.shape)
plt.hist(np.maximum(0,sigma.ravel()), log=True)
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'detach'

In [ ]:
import mcubes

threshold = 50.
print('fraction occupied', np.mean(sigma > threshold))
vertices, triangles = mcubes.marching_cubes(sigma, threshold)
print('done', vertices.shape, triangles.shape)

In [ ]:
maxs = torch.max(dataset.bbox[0], dim=0).values.numpy()
mins = torch.min(dataset.bbox[0], dim=0).values.numpy()

In [ ]:
vertices_n = vertices / L
for i in range(3):
    vertices_n[:,i] = vertices_n[:,i]*(maxs[i]-mins[i]) + mins[i]

In [ ]:
mesh = trimesh.Trimesh(vertices_n , triangles)
mesh.show()

In [ ]:
export = mesh.export(file_type='obj')

In [ ]:
with open('xzq.obj','w') as f:
    f.write(export)

In [23]:
pts = raw_xyz[sigma[:,:,:]>50,:]

In [25]:
np.savetxt('pts.txt',pts)